# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [1]:
import pandas as pd
import requests
import json

In [2]:
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
    with open('config.json.sample','r') as f:
        config = json.load(f)
    
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    
    with open('config.json','w+') as f:
        json.dump(config, f)

# Fetches variables: user_agent, key1, key2, key3

# Fetching Data from Last.fm

In [1]:
# This function gets the "Pages" where each page records 200 songs.
def getScrobbles(username='snaekboi', page=1, limit=200):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':username,
    'page': page,
    'api_key': API_KEY,
    'method': 'user.getrecenttracks',
    'format': 'json'
    }
    
    reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return reply

In [2]:
# This function takes the page number as the input and fetches the scrobbnles on that page.
def getPage(num=1, username='snaekboi'):
    response=getScrobbles(username, num)
    df = pd.DataFrame.from_dict(response.json(), orient='index')
    
    if type(df.track[0])==dict:
        ls = [df.track[0]]
    else:
        ls = df.track[0]
    page = pd.DataFrame.from_dict(ls)
    if (num!=1) & (page['@attr'][0]=={'nowplaying': 'true'}):
        page = page.drop([0])
    return page

In [ ]:
# Under Construction!